# Dell Qiskit Runtime Emulator Examples

# Local Execution
    
The following program walks through a (simple) example usage of the 
Qiskit Runtime Emulator in a local execution environment: i.e. potentially
using a locally installed simulator or a remote call directly from a
local machine to a remote simulator or QPU.

In [ ]:
from qiskit_emulator import EmulatorProvider
from qiskit import QuantumCircuit
import logging
import requests
import time
import os


If the program that interacts with the simulator/QPU is small enough,
it can be stored as a string in the file that interfaces with the 
provider. Both directories and files can be taken as input, as well.

In [ ]:
RUNTIME_PROGRAM = """
# This code is part of qiskit-runtime.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):
    circuits = transpile(
        circuits,
    )
    
    user_messenger.publish({'results': 'intermittently'}, final=False)

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results using either simulator or QPU backend.
    result = backend.run(circuits, **kwargs).result()

    user_messenger.publish(result.to_dict(), final=True)
"""

RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

PROGRAM_PREFIX = 'qiskit-test'
REMOTE_RUNTIME = os.getenv("SERVER_URL") 

logging.basicConfig(level=logging.DEBUG)

The EmulatorProvider is an interface that offers a choice of runtime
(local or remote) and lists which backends are available locally.

Both the client process and the job process interact with the 
EmulatorProvider - the client mainly uses it for the runtime while the 
job process selects a backend from the list.

In [ ]:
provider = EmulatorProvider()

The runtime is a service provided that allows clients to upload, update,
view, and run programs inside an execution environment. Since the client
has not specified a remote runtime to the provider it defaults to local.

In [ ]:
program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)
print(f"PROGRAM ID: {program_id}")
programs = provider.runtime.pprint_programs(refresh=True)

The following updates the existing program with a new description - this can be done for any of the metadata fields or the program data itself, though changes to the program data are not shown in the `pprint_programs` output.

In [ ]:
provider.runtime.update_program(program_id, description="IBM/Dell Updated Qiskit Runtime Program")
programs = provider.runtime.pprint_programs(refresh=True)

Below we use the Qiskit QuantumCircuit to create a circuit for our program to run. We then place that circuit in `program_inputs` - a dictionary of things that will be provided to our runtime program.

In [ ]:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

Through the `provider` we are able to run an instance of our program with the inputs we have created. 

When we run a job locally, a new process is started. This new process returns results to the main process via a socket connection.

In [ ]:
job = provider.runtime.run(program_id, options=None, inputs=program_inputs)

We can obtain a job's final results and specify a timeout for how long we are willing to wait. If no timeout is specified, the function will return `None` or the final results if they are present.

In [ ]:
results = job.result(timeout=60)
print(results)

We can also provide a callback function to the runtime for a job. A thread launched in the client process to poll for messages will call the callback when a non-final message is received.

In [ ]:
def callback_function(msg):
    print(f'******************\n\n\nFrom Callback Function: {msg}\n\n\n******************')

job = provider.runtime.run(program_id, inputs=program_inputs, options=None, callback=callback_function)

You may also specify a different backend on which you would like the quantum code to run. The default backend is the Qiskit Aer simulator.

In [ ]:
program_inputs['backend_name'] = 'emulator'
job = provider.runtime.run(program_id, inputs=program_inputs, options=None, callback=callback_function)